<a href="https://colab.research.google.com/github/marufahmed/datatools/blob/main/IOU_comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
# Individual overlap
import json

# Function to convert coordinates to pixel values for 2480 × 3508 resolution
def convert_to_pixels(coordinates, original_width, original_height):
    top_left_x, top_left_y = coordinates[0]
    bottom_right_x, bottom_right_y = coordinates[1]

    converted_top_left_x = int((top_left_x / original_width) * 2480)
    converted_top_left_y = int((top_left_y / original_height) * 3508)
    converted_bottom_right_x = int((bottom_right_x / original_width) * 2480)
    converted_bottom_right_y = int((bottom_right_y / original_height) * 3508)

    return (converted_top_left_x, converted_top_left_y), (converted_bottom_right_x, converted_bottom_right_y)

# Function to calculate area of a rectangle given its coordinates
def calculate_area(points):
    top_left_x, top_left_y = points[0]
    bottom_right_x, bottom_right_y = points[1]

    area = abs((bottom_right_x - top_left_x) * (bottom_right_y - top_left_y))
    return area

# Function to calculate IoU
def calculate_iou(gt_areas, detected_areas, threshold=0.5):
    iou_values = []

    for idx_gt, gt_area in enumerate(gt_areas):
        iou_for_gt = []

        for idx_det, det_area in enumerate(detected_areas):
            intersection = min(gt_area, det_area)
            union = gt_area + det_area - intersection

            iou = intersection / union if union > 0 else 0
            iou_for_gt.append(iou)

        max_iou_for_gt = max(iou_for_gt) if iou_for_gt else 0
        if max_iou_for_gt >= threshold:
            iou_values.append(max_iou_for_gt)

    return iou_values

# Read Ground Truth JSON and convert coordinates to pixel values
with open('/Users/marufahmed/Work/Machine Learning/WACV/resized_images/Gujrati/ground_truth.json', 'r') as file:
    ground_truth_data = json.load(file)
    original_width = 2480  # Replace with the original width of your image
    original_height = 3508  # Replace with the original height of your image
    ground_truth_areas = []

    for shape in ground_truth_data['shapes']:
        converted_points = convert_to_pixels(shape['points'], original_width, original_height)
        area = calculate_area(converted_points)
        ground_truth_areas.append(area)

# Read Detected Annotations JSON and convert coordinates to pixel values
with open('/Users/marufahmed/Work/Machine Learning/WACV/resized_images/Gujrati/llm.json', 'r') as file:
    detected_data = json.load(file)
    detected_areas = []

    for shape in detected_data['shapes']:
        converted_points = convert_to_pixels(shape['points'], original_width, original_height)
        area = calculate_area(converted_points)
        detected_areas.append(area)

## Classify coordinates based on labels if available
ground_truth_classified = {'paragraph': [], 'table': [], 'image': []}
detected_classified = {'paragraph': [], 'table': [], 'image': []}

for idx, shape in enumerate(ground_truth_data['shapes']):
    label = shape.get('label')
    if label and f'{label.lower()}' in ground_truth_classified:
        ground_truth_classified[f'{label.lower()}'].append(ground_truth_areas[idx])

for idx, shape in enumerate(detected_data['shapes']):
    label = shape.get('label')
    if label and f'{label.lower()}' in detected_classified:
        detected_classified[f'{label.lower()}'].append(detected_areas[idx])

# Print the number of areas overlapping each class and the IoU for each class
for label, gt_areas in ground_truth_classified.items():
    detected_areas = detected_classified[label]
    num_overlapping_areas = len([iou for iou in calculate_iou(gt_areas, detected_areas) if iou >= 0.5])
    avg_iou = sum(calculate_iou(gt_areas, detected_areas)) / len(gt_areas) if len(gt_areas) > 0 else 0

    print(f"Class: {label}")
    print(f"Number of overlapping areas: {num_overlapping_areas}")
    print(f"Average IoU for {label}: {avg_iou}")
    print("----------")

Class: paragraph
Number of overlapping areas: 16
Average IoU for paragraph: 0.8432201356596237
----------
Class: table
Number of overlapping areas: 0
Average IoU for table: 0
----------
Class: image
Number of overlapping areas: 2
Average IoU for image: 0.7986962616857671
----------


In [2]:
# Collective overlap
import json

# Function to convert coordinates to pixel values for 2480 × 3508 resolution
def convert_to_pixels(coordinates, original_width, original_height):
    top_left_x, top_left_y = coordinates[0]
    bottom_right_x, bottom_right_y = coordinates[1]

    converted_top_left_x = int((top_left_x / original_width) * 2480)
    converted_top_left_y = int((top_left_y / original_height) * 3508)
    converted_bottom_right_x = int((bottom_right_x / original_width) * 2480)
    converted_bottom_right_y = int((bottom_right_y / original_height) * 3508)

    return (converted_top_left_x, converted_top_left_y), (converted_bottom_right_x, converted_bottom_right_y)

# Function to calculate area of a rectangle given its coordinates
def calculate_area(points):
    top_left_x, top_left_y = points[0]
    bottom_right_x, bottom_right_y = points[1]

    area = abs((bottom_right_x - top_left_x) * (bottom_right_y - top_left_y))
    return area

# Function to calculate IoU with collective overlap
def calculate_collective_overlap(gt_area, detected_areas):
    combined_detected_area = sum(detected_areas)
    intersection = min(gt_area, combined_detected_area)
    union = gt_area + combined_detected_area - intersection

    return intersection, union

# Updated IoU calculation
def calculate_iou_with_collective_overlap(gt_areas, detected_areas, threshold=0.5):
    iou_values = []

    for gt_area in gt_areas:
        overlapping_detected_areas = [det_area for det_area in detected_areas if calculate_iou([gt_area], [det_area])]
        if not overlapping_detected_areas:
            continue

        intersection, union = calculate_collective_overlap(gt_area, overlapping_detected_areas)

        iou = intersection / union if union > 0 else 0
        if iou >= threshold:
            iou_values.append(iou)

    return iou_values

  # Read Ground Truth JSON and convert coordinates to pixel values
with open('/Users/marufahmed/Work/Machine Learning/WACV/resized_images/Gujrati/ground_truth.json', 'r') as file:
    ground_truth_data = json.load(file)
    original_width = 2480  # Replace with the original width of your image
    original_height = 3508  # Replace with the original height of your image
    ground_truth_areas = []

    for shape in ground_truth_data['shapes']:
        converted_points = convert_to_pixels(shape['points'], original_width, original_height)
        area = calculate_area(converted_points)
        ground_truth_areas.append(area)

# Read Detected Annotations JSON and convert coordinates to pixel values
with open('yolop1.json', 'r') as file:
    detected_data = json.load(file)
    detected_areas = []

    for shape in detected_data['shapes']:
        converted_points = convert_to_pixels(shape['points'], original_width, original_height)
        area = calculate_area(converted_points)
        detected_areas.append(area)

## Classify coordinates based on labels if available
ground_truth_classified = {'paragraph': [], 'table': [], 'image': []}
detected_classified = {'paragraph': [], 'table': [], 'image': []}

for idx, shape in enumerate(ground_truth_data['shapes']):
    label = shape.get('label')
    if label and f'{label.lower()}' in ground_truth_classified:
        ground_truth_classified[f'{label.lower()}'].append(ground_truth_areas[idx])

for idx, shape in enumerate(detected_data['shapes']):
    label = shape.get('label')
    if label and f'{label.lower()}' in detected_classified:
        detected_classified[f'{label.lower()}'].append(detected_areas[idx])

# Print the number of areas overlapping each class and the IoU for each class
for label, gt_areas in ground_truth_classified.items():
    detected_areas = detected_classified[label]
    num_overlapping_areas = len([iou for iou in calculate_iou_with_collective_overlap(gt_areas, detected_areas) if iou >= 0.5])
    avg_iou = sum(calculate_iou_with_collective_overlap(gt_areas, detected_areas)) / len(gt_areas) if len(gt_areas) > 0 else 0

    print(f"Class: {label}")
    print(f"Number of overlapping areas: {num_overlapping_areas}")
    print(f"Average IoU for {label}: {avg_iou}")
    print("----------")

FileNotFoundError: [Errno 2] No such file or directory: 'gt1.json'